# Keras - Transfer Learning

based on https://www.youtube.com/watch?v=qFJeN9V1ZsI&t=6097s (DeepLizard tutorial)

## Download pre-trained model (VGG16)

In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import shutil
import itertools

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__)

TF version: 2.8.0


In [4]:
phisical_devices = tf.config.list_physical_devices('GPU')
print("GPU is", "available" if len(phisical_devices) > 0 else "NOT AVAILABLE")

GPU is available


2022-03-23 18:04:34.103764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 18:04:34.161534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 18:04:34.161698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [5]:
model_vgg16 = keras.applications.vgg16.VGG16()

2022-03-23 18:04:41.956977: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 18:04:41.957543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 18:04:41.957779: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 18:04:41.957935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [6]:
model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

#### Based on the original model let's build a new model but without the last layer

In [7]:
model_vgg16_modified = Sequential(name='vgg16_modified')
for layer in model_vgg16.layers[:-1] :
    layer.trainable = False
    model_vgg16_modified.add(layer)

In [8]:
model_vgg16_modified.add(Dense(units=2, activation='softmax'))

In [9]:
model_vgg16_modified.summary()

Model: "vgg16_modified"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)    

In [10]:
model_vgg16_modified.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

## Train the new model

In [11]:
len(model_vgg16_modified.layers)

22

In [4]:
train_path = 'data/cats_vs_dogs/train'
valid_path = 'data/cats_vs_dogs/valid'

In [13]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cats', 'dogs'],)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cats', 'dogs'], batch_size=20)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
model_vgg16_modified.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

Epoch 1/10


2022-03-23 18:05:07.838070: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-03-23 18:05:09.530676: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-23 18:05:10.282835: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.66GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-23 18:05:10.326977: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory

100/100 - 41s - loss: 0.3701 - accuracy: 0.8415 - val_loss: 0.1189 - val_accuracy: 0.9650 - 41s/epoch - 407ms/step
Epoch 2/10
100/100 - 34s - loss: 0.0912 - accuracy: 0.9680 - val_loss: 0.0822 - val_accuracy: 0.9730 - 34s/epoch - 341ms/step
Epoch 3/10
100/100 - 35s - loss: 0.0654 - accuracy: 0.9785 - val_loss: 0.0596 - val_accuracy: 0.9810 - 35s/epoch - 346ms/step
Epoch 4/10
100/100 - 34s - loss: 0.0521 - accuracy: 0.9830 - val_loss: 0.0593 - val_accuracy: 0.9800 - 34s/epoch - 340ms/step
Epoch 5/10
100/100 - 34s - loss: 0.0426 - accuracy: 0.9840 - val_loss: 0.0482 - val_accuracy: 0.9820 - 34s/epoch - 340ms/step
Epoch 6/10
100/100 - 39s - loss: 0.0350 - accuracy: 0.9895 - val_loss: 0.0475 - val_accuracy: 0.9830 - 39s/epoch - 385ms/step
Epoch 7/10
100/100 - 36s - loss: 0.0297 - accuracy: 0.9920 - val_loss: 0.0439 - val_accuracy: 0.9830 - 36s/epoch - 362ms/step
Epoch 8/10
100/100 - 35s - loss: 0.0255 - accuracy: 0.9930 - val_loss: 0.0451 - val_accuracy: 0.9840 - 35s/epoch - 349ms/step
Epo

In [24]:
type(model_vgg16_modified.history.history)

dict

In [15]:
predictions = model_vgg16_modified.predict(x=valid_batches, verbose=2)
predictions[:10]

50/50 - 11s - 11s/epoch - 220ms/step


array([[1.4497005e-05, 9.9998546e-01],
       [9.9750561e-01, 2.4944679e-03],
       [3.7032361e-03, 9.9629682e-01],
       [9.9912375e-01, 8.7627303e-04],
       [9.9695587e-01, 3.0441179e-03],
       [1.7036638e-03, 9.9829632e-01],
       [3.6273373e-03, 9.9637270e-01],
       [5.3146683e-02, 9.4685328e-01],
       [9.9875069e-01, 1.2492965e-03],
       [5.9982989e-07, 9.9999940e-01]], dtype=float32)

In [16]:
rounded_predictions = np.round(predictions)
print(rounded_predictions.shape)
rounded_predictions[:10]

(1000, 2)


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [17]:
predicted_labels = np.argmax(rounded_predictions, axis=1)
predicted_labels[:10]

array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1])

In [18]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib

In [19]:
# ConfusionMatrixDisplay.from_predictions(y_true=valid_batches.classes, y_pred=predicted_labels) this doea not work with conda ml2 env (missing matplotlib)

ImportError: ConfusionMatrixDisplay.from_predictions requires matplotlib. You can install matplotlib with `pip install matplotlib`